## **The issue of pattern recognition (Electroencephalogram Signals)**

##### **AI and Biological Computations project**

##### **Tina Halimi**


In [1]:
import numpy as np
import scipy.io
from scipy.io import loadmat
from scipy.signal import welch
from scipy.integrate import simps
from scipy.signal import periodogram
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import ParameterGrid
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
data = loadmat('Project_data.mat')

In [3]:
# extracting data
data_channels = data['Channels']
data_fs = data['fs']
training_data = data['TrainData']
training_data_labels = data['TrainLabels']
test_data = data['TestData']

training_data_pos = training_data[:,:, training_data_labels[0,:] == 1]
training_data_neg = training_data[:, :, training_data_labels[0,:] == -1]

print(training_data.shape)
print(training_data_labels.shape)
print(training_data_pos.shape)
print(training_data_neg.shape)
print(test_data.shape)

(59, 5000, 550)
(1, 550)
(59, 5000, 277)
(59, 5000, 273)
(59, 5000, 159)


## phase 1

### Fisher 

In [4]:
def fisher1D(mean_total,mean_positive,mean_negative,var_positive,var_negative):
    sb = (mean_positive - mean_total)**2 + (mean_negative - mean_total) **2
    sw = var_positive + var_negative
    return sb/sw

In [5]:
def min_max_normalize(input_array):

    scaler = MinMaxScaler()
    transposed_array = np.transpose(input_array)
    normalized_array_transposed = scaler.fit_transform(transposed_array)
    normalized_array = np.transpose(normalized_array_transposed)

    return normalized_array


### Features of the frequency domain

In [6]:
fishersOfFeatures_freq=[]

#### **Maximum Frequency**

In [7]:
# function
def max_power_frequency(data, sampling_rate):

    channels, samples, experiments = data.shape
    max_power_freqs = np.zeros((channels,experiments))
    
    for channel in range(channels):
        for experiment in range(experiments):

            signal = data[channel, :, experiment]
            frequencies, power_spectrum = periodogram(signal, sampling_rate)
            frequencies = frequencies.ravel()
            max_power_freq = np.argmax(power_spectrum)

            max_power_freqs[channel, experiment] = frequencies[max_power_freq]

    return np.array(max_power_freqs)


In [8]:
#result
max_freq = max_power_frequency(training_data,data_fs)
print(max_freq.shape)

(59, 550)


In [9]:
# fisher calculation
mean_total_max = np.mean(max_freq,axis=1)

mean_pos_max = np.mean(max_freq[:, training_data_labels[0,:] == 1],axis = 1)
mean_neg_max = np.mean(max_freq[:, training_data_labels[0,:] == -1],axis = 1)

var_pos_max = np.var(max_freq[:, training_data_labels[0,:] == 1],axis = 1)
var_neg_max = np.var(max_freq[:, training_data_labels[0,:] == -1],axis = 1)

fisher_max_freq = fisher1D(mean_total_max, mean_pos_max, mean_neg_max, var_pos_max, var_neg_max)

print(fisher_max_freq.shape)
print('fisher of Max Frequency feature: \n')
print(fisher_max_freq)


(59,)
fisher of Max Frequency feature: 

[1.11233581e-03 2.62065453e-05 2.66865597e-04 8.97682267e-04
 4.22944460e-03 3.43219286e-04 3.02999010e-03 7.56429718e-03
 2.08692441e-03 1.75142659e-04 1.82403612e-04 1.49703841e-03
 5.26426672e-04 4.91131275e-03 1.00365235e-04 2.52625157e-03
 3.03125545e-03 3.40073486e-03 3.79370916e-03 4.47843588e-04
 1.02628641e-02 3.85509772e-03 2.25118113e-02 2.37471860e-02
 1.62021549e-03 9.79055792e-05 1.57655824e-03 4.65118305e-06
 1.88925159e-04 8.92823758e-03 4.89029084e-04 3.70838689e-03
 3.78918506e-03 6.92575222e-03 5.92031027e-03 9.35608226e-04
 3.56346412e-03 1.44063007e-03 3.86798230e-03 1.40852317e-02
 4.35856010e-07 4.51917727e-07 2.35174138e-02 2.02593869e-04
 4.65156222e-04 1.34994870e-06 3.26188476e-04 2.72353198e-03
 1.22206326e-03 6.32949115e-03 1.67284617e-02 4.28974610e-04
 1.65959464e-04 1.71965384e-02 1.29976269e-03 1.59202326e-03
 1.07452552e-03 3.54510545e-03 6.64270426e-04]


In [10]:
max_freq_test = max_power_frequency(test_data, data_fs)
print(max_freq_test.shape)

(59, 159)


In [11]:
for i in range(len(fisher_max_freq)):
    fishersOfFeatures_freq.append([fisher_max_freq[i],'max_freq_channel' + str(i), max_freq[i],max_freq_test[i]])

print(len(fishersOfFeatures_freq))

59


#### **Mean Frequency**

In [12]:
# function
def weighted_average_frequency(data, sampling_rate):
    channels, samples, experiments = data.shape
    weighted_avg_freqs = np.zeros((channels, experiments))
    
    for channel in range(channels):
        for experiment in range(experiments):
            signal = data[channel, :, experiment]
            frequencies, power_spectrum = periodogram(signal, sampling_rate)
            frequencies = frequencies.ravel()
            
            weighted_avg_freq = np.sum(abs(frequencies) * power_spectrum) / np.sum(power_spectrum)
            weighted_avg_freqs[channel, experiment] = weighted_avg_freq

    return np.array(weighted_avg_freqs)


In [13]:
# result
mean_freq = weighted_average_frequency(training_data,data_fs)
print(mean_freq.shape)


(59, 550)


In [14]:
# fisher
mean_total_mean = np.mean(mean_freq,axis=1)

mean_pos_mean = np.mean(mean_freq[:, training_data_labels[0,:] == 1],axis = 1)
mean_neg_mean = np.mean(mean_freq[:, training_data_labels[0,:] == -1],axis = 1)

var_pos_mean = np.var(mean_freq[:, training_data_labels[0,:] == 1],axis = 1)
var_neg_mean = np.var(mean_freq[:, training_data_labels[0,:] == -1],axis = 1)

fisher_mean_freq = fisher1D(mean_total_mean, mean_pos_mean, mean_neg_mean, var_pos_mean, var_neg_mean)

print(fisher_mean_freq.shape)
print('fisher of Mean Frequency feature: \n')
print(fisher_mean_freq)

(59,)
fisher of Mean Frequency feature: 

[3.53226012e-05 8.93360396e-04 9.32510044e-04 7.16705637e-06
 1.30270756e-03 4.71071123e-03 2.23836044e-03 2.24672617e-04
 1.53898343e-04 5.99652301e-05 2.54376013e-07 2.41280754e-04
 3.89640234e-03 1.05173714e-03 2.87656093e-04 1.50652372e-03
 5.17532004e-03 3.45501614e-04 4.18881783e-05 5.32379879e-03
 2.88791770e-03 7.94393856e-03 2.45326741e-02 4.75426259e-03
 1.55103874e-03 8.15722497e-03 3.49896211e-04 4.74848728e-05
 1.38166870e-03 3.00275167e-03 1.08842566e-02 9.27766963e-03
 1.88780801e-02 6.02426232e-03 3.57959154e-04 2.48179391e-03
 6.21576126e-04 1.66573425e-03 6.00784184e-03 6.25403729e-03
 2.38945444e-03 6.02398205e-03 1.09977381e-02 2.85530763e-03
 1.30783818e-03 1.51143951e-03 8.22380003e-03 2.84242289e-03
 8.74431885e-03 7.04702004e-03 7.77159156e-04 8.67835838e-04
 8.66077511e-03 1.03330276e-02 3.81632883e-03 1.44130305e-02
 6.80853457e-03 2.79089758e-05 1.79165018e-02]


In [15]:
mean_freq_test = weighted_average_frequency(test_data,data_fs)
print(mean_freq_test.shape)

(59, 159)


In [16]:
for i in range(len(fisher_mean_freq)):
    fishersOfFeatures_freq.append([fisher_mean_freq[i],'mean_freq_channel' + str(i), mean_freq[i], mean_freq_test[i]])

print(len(fishersOfFeatures_freq))

118


#### **Median Frequency**

In [17]:
# function
def weighted_median_frequency(data, sampling_rate):

    channels, samples, experiments = data.shape
    weighted_median_freqs = np.zeros((channels, experiments))
    
    for channel in range(channels):
        for experiment in range(experiments):

            signal = data[channel, :, experiment]
            frequencies, power_spectrum = periodogram(signal, sampling_rate)
            frequencies = frequencies.ravel()
            cumsum_power = np.cumsum(power_spectrum)

            median_index = np.argmax(cumsum_power >= 0.5 * np.sum(power_spectrum))
            
            weighted_median_freq = frequencies[median_index]

            weighted_median_freqs[channel, experiment] = weighted_median_freq

    return np.array(weighted_median_freqs)


In [18]:
# result
mod_freq = weighted_median_frequency(training_data,data_fs)
print(mod_freq.shape)


(59, 550)


In [19]:
# fisher
mean_total_mod = np.mean(mod_freq,axis=1)

mean_pos_mod = np.mean(mod_freq[:, training_data_labels[0,:] == 1],axis = 1)
mean_neg_mod = np.mean(mod_freq[:, training_data_labels[0,:] == -1],axis = 1)

var_pos_mod = np.var(mod_freq[:, training_data_labels[0,:] == 1],axis = 1)
var_neg_mod = np.var(mod_freq[:, training_data_labels[0,:] == -1],axis = 1)

fisher_mod_freq = fisher1D(mean_total_mod, mean_pos_mod, mean_neg_mod, var_pos_mod, var_neg_mod)

print(fisher_mod_freq.shape)

print('fisher of Median Frequency feature: \n')
print(fisher_mod_freq)

(59,)
fisher of Median Frequency feature: 

[1.70350000e-05 2.66159443e-04 8.17056023e-04 1.03030082e-05
 5.57736672e-04 2.05300587e-03 5.66141088e-04 5.22094089e-04
 8.33319101e-05 1.68327694e-05 9.10330147e-05 1.78272409e-04
 4.84309714e-03 3.40514427e-03 2.64873591e-04 1.35180765e-03
 5.48515509e-03 2.22436934e-04 2.44490062e-04 5.40259701e-03
 5.10851195e-03 8.62512061e-03 2.02976836e-02 5.45537910e-03
 2.82773707e-04 1.17719748e-02 5.00529949e-04 2.78538753e-04
 8.78699206e-06 1.22012453e-03 9.72575315e-03 5.41049596e-03
 1.96915569e-02 2.80893753e-03 4.45801945e-03 7.20958342e-04
 1.26119549e-05 3.51281871e-03 2.91636439e-03 5.53695715e-03
 9.85639124e-04 7.49386657e-03 3.13017095e-02 6.75271616e-04
 2.57170841e-03 9.90775419e-04 1.63672158e-02 2.64822522e-03
 9.93946378e-03 4.50970681e-03 1.24509000e-03 9.94851362e-05
 1.04146489e-02 1.94346094e-02 9.38907969e-03 2.01237495e-02
 6.68056696e-03 1.04239468e-04 1.95441342e-02]


In [20]:
mod_freq_test = weighted_median_frequency(test_data,data_fs)
print(mod_freq_test.shape)

(59, 159)


In [21]:
for i in range(len(fisher_mod_freq)):
    fishersOfFeatures_freq.append([fisher_mod_freq[i],'mod_freq_channel' + str(i), mod_freq[i],mod_freq_test[i]])

print(len(fishersOfFeatures_freq))

177


#### **Relative Energy**

In [22]:
def calculate_relative_band_energies(data, sampling_rate, bands):
    channels, samples, experiments = data.shape

    relative_band_energies = np.zeros((channels, experiments, len(bands.keys())))
    band_energies = np.zeros((channels, experiments, len(bands.keys())))

    for channel in range(channels):
        for experiment in range(experiments):
            signal = data[channel, :, experiment]
            frequencies, psd = periodogram(signal, sampling_rate)
            frequencies = frequencies.ravel()

            total_energy = 0
            for k, (band, (low, high)) in enumerate(bands.items()):
                band_energy = 0
                for i, freq in enumerate(frequencies):
                    if (low <= freq) and (freq < high):
                        band_energy += psd[i]
                # idx_band = np.logical_and(frequencies >= low, frequencies <= high)
                # band_energy = simps(psd[idx_band], frequencies[idx_band])
                total_energy += band_energy
                band_energies[channel, experiment, k] = band_energy
            #print(total_energy)
            relative_band_energies[channel, experiment, :] = band_energies[channel, experiment, :] / total_energy
    
    return relative_band_energies

In [23]:
# result
bands = {
    'delta': [0.1, 3],
    'theta': [4, 7],
    'alpha': [8, 12],
    'low_range_beta': [12, 15],
    'mid_range_beta': [16, 20],
    'high_range_beta': [21, 30],
    'gamma': [30, 100]
}

bands2 = {
    'theta': [4, 8],
    'alpha': [8, 13],
    'beta': [13, 30],
    'gamma': [30, 49],
}

relative_band_energies = calculate_relative_band_energies(training_data, data_fs, bands2)

In [24]:
print(relative_band_energies.shape)

(59, 550, 4)


In [25]:
mean_total_bands = np.mean(relative_band_energies, axis=1)

mean_pos_bands = np.mean(relative_band_energies[:, training_data_labels[0,:] == 1],axis = 1)
mean_neg_bands = np.mean(relative_band_energies[:, training_data_labels[0,:] == -1],axis = 1)

var_pos_bands = np.var(relative_band_energies[:, training_data_labels[0,:] == 1],axis = 1)
var_neg_bands = np.var(relative_band_energies[:, training_data_labels[0,:] == -1],axis = 1)

fishers_of_bands = fisher1D(mean_total_bands,mean_pos_bands,mean_neg_bands,var_pos_bands,var_neg_bands)

print(fishers_of_bands.shape)
print('fisher of Relative Band Energies feature: \n')
print('theta:\n')
print(fishers_of_bands[:,0],4)
print('\n alpha \n')
print(fishers_of_bands[:,1],4)


(59, 4)
fisher of Relative Band Energies feature: 

theta:

[2.04990726e-04 2.77638387e-03 5.65286397e-05 1.00647150e-03
 3.38405292e-03 4.20832211e-03 1.38152897e-03 1.13261368e-03
 3.34775812e-05 5.34870001e-03 1.17443235e-04 1.25841696e-03
 7.75541728e-03 3.30837852e-04 4.29195252e-03 2.28281959e-04
 9.92353460e-03 6.79822145e-04 7.64690613e-03 4.28266601e-03
 7.99432578e-04 2.21006690e-03 2.47960593e-02 5.99876581e-03
 7.15064029e-03 5.31548193e-03 2.71210686e-03 4.37206906e-03
 4.43538668e-04 3.29791344e-05 1.13148223e-02 1.03293511e-02
 1.12782376e-02 1.08728454e-02 1.64732741e-03 2.05692241e-03
 4.77074373e-05 8.69449889e-04 5.09948088e-03 9.04204938e-03
 6.34475131e-03 1.63645632e-02 1.40027700e-03 8.87825355e-06
 6.24138735e-04 2.50192581e-03 1.13956253e-02 8.99459433e-03
 1.34320277e-02 3.58258866e-02 6.68065033e-03 1.73407485e-02
 7.70425354e-03 9.48050085e-03 9.98689703e-03 1.17017396e-02
 3.56726767e-02 7.80340334e-03 2.54916964e-02] 4

 alpha 

[1.39490121e-03 2.62251980e

In [26]:
print('beta \n')
print(fishers_of_bands[:,2],4)
print('\n gamma \n')
print(fishers_of_bands[:,3],4)


beta 

[1.04491625e-03 3.91718961e-03 2.90566667e-06 4.49437385e-03
 4.21931273e-03 3.31507947e-04 6.52252275e-03 4.17220737e-04
 3.05606908e-05 2.54709614e-03 3.43327176e-04 1.72036371e-03
 4.29275005e-03 4.61738909e-05 2.13617726e-03 4.37921632e-04
 2.43834575e-05 3.54495995e-04 4.63620367e-04 1.59339213e-03
 2.28839324e-03 2.33099141e-03 1.28032098e-02 4.81051792e-03
 8.23807883e-03 4.63251722e-03 2.10473362e-03 5.47836938e-03
 2.60365500e-03 6.82453791e-03 5.01067120e-03 1.05156908e-02
 9.18372660e-04 7.32026066e-03 2.43773347e-03 7.59116589e-03
 1.85380839e-03 5.89063337e-03 1.18901875e-03 7.77085630e-03
 1.46203068e-02 6.08266594e-03 1.49195269e-02 5.34393390e-04
 4.56079525e-03 2.07909127e-04 3.60760213e-03 2.12751028e-03
 5.47183011e-03 1.46954054e-02 9.00865743e-03 1.23421875e-02
 2.98743664e-03 2.33028868e-03 4.99177250e-03 3.19589326e-03
 1.45837396e-02 8.33019152e-05 7.25625283e-03] 4

 gamma 

[1.79062119e-04 2.79029471e-04 6.01604128e-05 1.46856990e-04
 1.51750836e-03 7.9

In [27]:
relative_band_energies_test = calculate_relative_band_energies(test_data, data_fs, bands2)
print(relative_band_energies_test.shape)

(59, 159, 4)


In [28]:
for i in range(fishers_of_bands.shape[0]):
        fishersOfFeatures_freq.append([fishers_of_bands[i,0],'band_theta_channel' + str(i), relative_band_energies[i,:,0],relative_band_energies_test[i,:,0]])
        fishersOfFeatures_freq.append([fishers_of_bands[i,1],'band_alpha_channel' + str(i), relative_band_energies[i,:,1], relative_band_energies_test[i,:,1]])
        fishersOfFeatures_freq.append([fishers_of_bands[i,2],'band_beta_channel' + str(i), relative_band_energies[i,:,2], relative_band_energies_test[i,:,2]])
        fishersOfFeatures_freq.append([fishers_of_bands[i,3],'band_gamma_channel' + str(i), relative_band_energies[i,:,3], relative_band_energies_test[i,:,3]])
        
print(len(fishersOfFeatures_freq))

413


## *Statistical features*

In [29]:
fishersOfFeatures_stat=[]

#### **Variance**

In [30]:
# function
def calc_variance(data):
    return np.var(data, axis=1)


variance = calc_variance(training_data)
print(variance.shape)

(59, 550)


In [31]:
# fisher
mean_total_var = np.mean(variance,axis=1)

mean_pos_var = np.mean(variance[:, training_data_labels[0,:] == 1],axis = 1)
mean_neg_var = np.mean(variance[:, training_data_labels[0,:] == -1],axis = 1)

var_pos_var = np.var(variance[:, training_data_labels[0,:] == 1],axis = 1)
var_neg_var = np.var(variance[:, training_data_labels[0,:] == -1],axis = 1)

fisher_var = fisher1D(mean_total_var, mean_pos_var, mean_neg_var, var_pos_var, var_neg_var)

print(fisher_var.shape)
print('fisher of variance feature: \n')
print(fisher_var)


(59,)
fisher of variance feature: 

[1.52394156e-06 1.73079774e-03 3.20643248e-03 8.81358431e-04
 6.06595201e-03 1.02018027e-02 1.04566148e-02 1.78090820e-03
 3.81505106e-03 7.37470031e-03 1.56928825e-03 4.83497522e-03
 1.32857437e-02 2.68406128e-03 1.24992780e-02 5.32735097e-03
 1.96545579e-03 3.94847633e-03 3.51609845e-03 2.97559573e-03
 1.79632502e-03 2.43538807e-04 3.38514735e-03 2.37443284e-03
 3.59120962e-03 8.35729020e-03 2.48960766e-03 3.75756308e-03
 2.11889684e-03 1.22284553e-03 3.24048063e-05 1.74225079e-03
 4.93023939e-03 2.98817088e-03 9.18157757e-03 8.53321215e-03
 4.77973271e-03 1.89556030e-03 7.63670189e-04 1.84095545e-04
 9.01492585e-04 4.02095217e-03 1.56557017e-02 6.87409816e-03
 1.58501039e-03 6.25736519e-03 1.38436511e-03 4.39152291e-03
 1.07094719e-04 1.07304448e-02 2.57599852e-03 5.31913495e-03
 1.12529830e-02 7.20925516e-08 4.74217456e-03 6.59729072e-05
 6.88301200e-03 5.03835631e-04 1.01854443e-03]


In [32]:
variance_test = calc_variance(test_data)
print(variance_test.shape)

(59, 159)


In [33]:
for i in range(len(fisher_var)):
    fishersOfFeatures_stat.append([fisher_var[i],'var_channel' + str(i), variance[i], variance_test[i]])

print(len(fishersOfFeatures_stat))

59


#### **Domain Histogram**

In [34]:
def calc_domain_histogram(data, num_bins):
    channels, samples, experiments = data.shape
    
    domain_histograms = np.zeros((channels, num_bins, experiments))
    
    for channel in range(channels):
        for experiment in range(experiments):     

            hist, bin_edges = np.histogram(data[channel, :, experiment], bins=num_bins)
            domain_histograms[channel, :, experiment] = hist

    return domain_histograms


In [35]:
#result
domain_histogram = calc_domain_histogram(training_data,10)
print(domain_histogram.shape)

(59, 10, 550)


In [36]:
# fisher
mean_total_hist = np.mean(domain_histogram, axis=2)

mean_pos_hist = np.mean(domain_histogram[:,:, training_data_labels[0,:] == 1],axis = 2)
mean_neg_hist = np.mean(domain_histogram[:,:, training_data_labels[0,:] == -1],axis = 2)

var_pos_hist = np.var(domain_histogram[:,:, training_data_labels[0,:] == 1],axis = 2)
var_neg_hist = np.var(domain_histogram[:,:, training_data_labels[0,:] == -1],axis = 2)

fishers_of_hist = fisher1D(mean_total_hist,mean_pos_hist,mean_neg_hist,var_pos_hist,var_neg_hist)

print(fishers_of_hist.shape)
print('fisher of Domain Histogram feature: \n')
print(fishers_of_hist)



(59, 10)
fisher of Domain Histogram feature: 

[[1.29935616e-02 9.81471574e-03 8.45622597e-03 5.21142527e-03
  3.72736191e-03 4.77256752e-03 6.81283913e-03 6.42069127e-03
  1.17336169e-02 1.19115927e-02]
 [2.21520998e-03 5.80796635e-03 7.50476161e-03 5.57822087e-03
  1.10114621e-03 5.30350044e-03 4.48305929e-03 3.12714166e-03
  3.00753607e-04 2.32287666e-04]
 [3.41007136e-03 9.02866637e-04 5.06643494e-05 1.09266894e-04
  9.28768631e-04 3.17958325e-03 3.10195645e-07 2.67175617e-03
  1.41378965e-03 3.51405462e-04]
 [5.66002181e-03 1.85408776e-08 9.78699669e-04 3.96401741e-03
  5.56655807e-03 2.70805846e-03 2.67844913e-03 2.76696715e-03
  2.91934029e-03 2.64540762e-03]
 [6.12088463e-05 6.47421414e-05 5.34258147e-04 4.40140542e-04
  4.91806137e-04 3.21722314e-03 1.01221586e-04 2.49638245e-04
  5.14710976e-05 4.13962237e-03]
 [1.25790667e-03 6.71163387e-03 7.87017357e-03 2.08864281e-03
  2.64534292e-04 8.97068129e-05 2.20842757e-03 4.24566770e-03
  4.07850965e-03 8.13954764e-04]
 [1.0891466

In [37]:
domain_histogram_test = calc_domain_histogram(test_data,10)
print(domain_histogram_test.shape)

(59, 10, 159)


In [38]:

for i in range(fishers_of_hist.shape[0]):
    for j in range(fishers_of_hist.shape[1]):
        fishersOfFeatures_stat.append([fishers_of_hist[i,j],'hist' + str(j) +'_channel'+ str(i) , domain_histogram[i,j,:], domain_histogram_test[i,j,:]])

print(len(fishersOfFeatures_stat))



649


#### **Form Factor**

In [39]:
# function
def calc_form_factor(data, sampling_rate):
    channels, samples, experiments = data.shape
    
    form_factors = np.zeros((channels, experiments))
    
    for channel in range(channels):
        for experiment in range(experiments):
        
            first_derivative = np.diff(data[channel, :, experiment])
            second_derivative = np.diff(first_derivative)
            
            std_signal = np.std(data[channel, :, experiment])
            std_first_derivative = np.std(first_derivative)
            std_second_derivative = np.std(second_derivative)

            ff = (std_second_derivative/std_first_derivative) / (std_first_derivative/std_signal)
            form_factors[channel, experiment] = ff

    return form_factors

In [40]:
#result
form_factor = calc_form_factor(training_data,data_fs)
print(form_factor.shape)

(59, 550)


In [41]:
# fisher
mean_total_ff = np.mean(form_factor,axis=1)

mean_pos_ff = np.mean(form_factor[:, training_data_labels[0,:] == 1],axis = 1)
mean_neg_ff = np.mean(form_factor[:, training_data_labels[0,:] == -1],axis = 1)

var_pos_ff = np.var(form_factor[:, training_data_labels[0,:] == 1],axis = 1)
var_neg_ff = np.var(form_factor[:, training_data_labels[0,:] == -1],axis = 1)

fisher_ff = fisher1D(mean_total_ff, mean_pos_ff, mean_neg_ff, var_pos_ff, var_neg_ff)

print(fisher_ff.shape)
print('fisher of Form Factor feature: \n')
print(fisher_ff)


(59,)
fisher of Form Factor feature: 

[2.28544002e-03 6.92775439e-04 5.00181169e-03 5.35410609e-04
 4.14559108e-04 2.71237855e-03 3.71384662e-04 7.63962990e-04
 7.80301283e-04 1.01016260e-04 5.50472538e-04 1.59748306e-05
 2.02963280e-03 1.60409035e-03 2.46702009e-04 6.36300726e-04
 1.98757238e-03 3.31860399e-05 1.99154294e-07 2.56081809e-04
 4.28873683e-05 5.61049448e-05 5.32897358e-03 4.55814741e-04
 2.05462411e-03 3.43034644e-03 6.63825440e-06 4.19425312e-05
 5.89605538e-07 1.74044674e-04 1.39326582e-03 2.57610482e-03
 6.26591048e-04 3.52371702e-03 2.58499065e-05 1.72514827e-03
 1.81088751e-03 2.45200995e-04 1.23967384e-03 1.64874448e-03
 2.44408568e-03 2.91331317e-03 3.78839489e-05 5.38698204e-04
 4.83917553e-05 8.48486094e-04 5.13522637e-04 8.97840336e-04
 1.82945719e-03 4.57830968e-04 9.51722663e-04 5.59180770e-04
 1.18403323e-03 5.31050728e-04 1.08461162e-05 6.45300913e-04
 1.96620706e-06 1.46144732e-03 4.88225932e-03]


In [42]:
form_factor_test = calc_form_factor(test_data,data_fs)
print(form_factor_test.shape)

(59, 159)


In [43]:
for i in range(len(fisher_ff)):
    fishersOfFeatures_stat.append([fisher_ff[i],'ff_channel' + str(i), form_factor[i], form_factor_test[i]])

print(len(fishersOfFeatures_stat))

708


#### **Correlation**

In [44]:
# function
def calc_correlation(data):
    channels, samples, experiments = data.shape
    correlations = np.zeros((channels, channels, experiments))

    for experiment in range(experiments):
        for i in range(channels):
            for j in range(i+1, channels):
                
                # Calculate the correlation coefficient between signals i and j
                correlation_coefficient = np.corrcoef(data[i, :, experiment], data[j, :, experiment])[0, 1]
                correlations[i, j, experiment] = correlation_coefficient

    return correlations

In [45]:
# result
correlation_matrices = calc_correlation(training_data)
print(correlation_matrices.shape)

(59, 59, 550)


In [46]:
correlation_matrices_test = calc_correlation(test_data)
print(correlation_matrices_test.shape)

(59, 59, 159)


In [47]:
# fisher
mean_total_corr = np.mean(correlation_matrices, axis=2)

mean_pos_corr = np.mean(correlation_matrices[:,:, training_data_labels[0,:] == 1],axis = 2)
mean_neg_corr = np.mean(correlation_matrices[:,:, training_data_labels[0,:] == -1],axis = 2)

var_pos_corr = np.var(correlation_matrices[:,:, training_data_labels[0,:] == 1],axis = 2)
var_neg_corr = np.var(correlation_matrices[:,:, training_data_labels[0,:] == -1],axis = 2)

matrix_fishers_of_corr = fisher1D(mean_total_corr, mean_pos_corr, mean_neg_corr, var_pos_corr, var_neg_corr)

print(matrix_fishers_of_corr.shape)


(59, 59)


/var/folders/w1/6xdwk4ls5nb4d1hy_mlzx5380000gn/T/ipykernel_2141/2606028922.py:4: RuntimeWarning: invalid value encountered in divide
  return sb/sw


In [48]:
for i in range(59):
    for j in range(i+1, 59):
        fishersOfFeatures_stat.append([matrix_fishers_of_corr[i,j], 'corr_'+'channel'+str(i)+'channel'+str(j), correlation_matrices[i,j,:], correlation_matrices_test[i,j,:]])

print(len(fishersOfFeatures_stat))

2419


In [49]:
print(len(fishersOfFeatures_stat))

2419


In [50]:
#combining fishers of freq and fishers of stat
fishersOfFeatures = fishersOfFeatures_freq + fishersOfFeatures_stat
print(len(fishersOfFeatures))

2832


# *MLP*

extracting 50 best features according to Fisher method

In [51]:
#print(type(fishersOfFeatures_freq[0]))
number_of_features = 50

sorted_Fisher_freq = sorted(fishersOfFeatures, key=lambda x: x[0], reverse=True)
best_Fishers = sorted_Fisher_freq[:number_of_features]

best_features = []
best_features_test = []
best_features_names = []

for sublist in best_Fishers:
    best_features_names.append((sublist[1]))
    best_features.append(sublist[2])
    best_features_test.append(sublist[3])

best_features = np.array(best_features)
best_features_test = np.array(best_features_test)


print(best_features.shape)
print(best_features_test.shape)

print(best_features_names)

(50, 550)
(50, 159)
['band_theta_channel49', 'band_theta_channel56', 'hist2_channel10', 'mod_freq_channel42', 'hist3_channel19', 'band_gamma_channel58', 'band_theta_channel58', 'hist5_channel10', 'band_gamma_channel49', 'band_theta_channel22', 'mean_freq_channel22', 'hist3_channel48', 'max_freq_channel23', 'hist2_channel8', 'hist6_channel19', 'max_freq_channel42', 'hist5_channel19', 'max_freq_channel22', 'corr_channel38channel58', 'hist6_channel48', 'hist1_channel10', 'hist2_channel48', 'corr_channel47channel51', 'corr_channel41channel55', 'mod_freq_channel22', 'corr_channel36channel58', 'mod_freq_channel55', 'hist3_channel8', 'mod_freq_channel32', 'mod_freq_channel58', 'mod_freq_channel53', 'hist6_channel8', 'hist3_channel10', 'mean_freq_channel32', 'band_gamma_channel22', 'corr_channel12channel52', 'band_gamma_channel41', 'corr_channel34channel57', 'mean_freq_channel58', 'hist5_channel8', 'band_gamma_channel55', 'band_theta_channel51', 'max_freq_channel53', 'band_gamma_channel32', 'm

In [52]:
# normalize best features
best_features_normalize = min_max_normalize(best_features)
best_features_test_normalize = min_max_normalize(best_features_test)
print(best_features_normalize.shape)

(50, 550)


In [53]:
# saving different models accuracies and their predicted values
all_best_accuracies_MLP = []
all_predicted_MLP = []

implement MLP network

In [54]:
# train_data: (500, 50), train_labels: (550, 1)
train_data = best_features_normalize.T
train_labels = training_data_labels.T
testing_data = best_features_test_normalize.T

# Convert labels from -1 to 0 for compatibility with "binary_crossentropy" loss
train_labels = (train_labels + 1) / 2

num_folds = 5
kf = KFold(n_splits=num_folds)

parameters = [[1,[32],['relu']],
              [2,[64,32],['relu','relu']],
              [3,[128,64,32],['relu','relu','sigmoid']],
              [3,[128,64,32],['relu','relu','relu']],
              [3,[32,16,8],['relu','relu','sigmoid']],
              [4,[128,63,32,16],['relu','relu','relu','relu']],
              [5,[32,64,128,64,32],['relu','relu','relu','relu','relu']]
              ]

def create_mlp_model(num_layers, neurons_per_layer, activation):
    model = Sequential()
    model.add(Dense(units=neurons_per_layer[0], activation=activation[0], input_dim=train_data.shape[1]))

    for i in range(1, num_layers - 1):
        model.add(Dense(units=neurons_per_layer[i], activation=activation[i]))

    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

best_accuracy = 0

for params in parameters:
    fold_accuracies = []
    model = create_mlp_model(params[0],params[1],params[2])
    print(f'Parameters: number of layers:{params[0]}, number of neurons in layers:{params[1]}, activation of layers:{params[2]}')
    for train_index, val_index in kf.split(train_data):
        X_train, X_val = train_data[train_index], train_data[val_index]
        y_train, y_val = train_labels[train_index], train_labels[val_index]

        
        model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=0)

        val_predictions = (model.predict(X_val) > 0.5).astype(int)
        accuracy = accuracy_score(y_val, val_predictions)
        fold_accuracies.append(accuracy)

    average_accuracy = np.mean(fold_accuracies)
    print(f'Average Cross-Validation Accuracy: {average_accuracy * 100:.2f}%')
    print('-' * 30)


    model.fit(train_data, train_labels, epochs=30, batch_size=32, verbose=0)

    test_predict = (model.predict(testing_data) > 0.5).astype(int)

    all_best_accuracies_MLP.append(average_accuracy)
    all_predicted_MLP.append(test_predict)

    if average_accuracy > best_accuracy:
        best_accuracy = average_accuracy
        best_params = params


Parameters: number of layers:1, number of neurons in layers:[32], activation of layers:['relu']
4/4 [==============================] - 0s 534us/step
Average Cross-Validation Accuracy: 73.09%
------------------------------
5/5 [==============================] - 0s 414us/step
Parameters: number of layers:2, number of neurons in layers:[64, 32], activation of layers:['relu', 'relu']
4/4 [==============================] - 0s 487us/step
Average Cross-Validation Accuracy: 79.09%
------------------------------
5/5 [==============================] - 0s 406us/step
Parameters: number of layers:3, number of neurons in layers:[128, 64, 32], activation of layers:['relu', 'relu', 'sigmoid']
4/4 [==============================] - 0s 493us/step
Average Cross-Validation Accuracy: 90.55%
------------------------------
5/5 [==============================] - 0s 445us/step
Parameters: number of layers:3, number of neurons in layers:[128, 64, 32], activation of layers:['relu', 'relu', 'relu']
4/4 [=========

In [55]:
print(f'Best MLP Network Configuration:')
print(f'Number of Layers: {best_params[0]}')
print(f'Neurons per Layer: {best_params[1]}')
print(f'Activation Function: {best_params[2]}')
print(f'Average Cross-Validation Accuracy: {best_accuracy * 100:.2f}%')

Best MLP Network Configuration:
Number of Layers: 4
Neurons per Layer: [128, 63, 32, 16]
Activation Function: ['relu', 'relu', 'relu', 'relu']
Average Cross-Validation Accuracy: 91.82%


In [56]:
print(len(all_best_accuracies_MLP))
print(len(all_predicted_MLP))

7
7


randomly choose 50 features from 75 best featues

In [57]:
#print(type(fishersOfFeatures_freq[0]))
import random

number_of_features_all = 75
number_of_features = 50
num_it = 3
sorted_Fisher_freq = sorted(fishersOfFeatures, key=lambda x: x[0], reverse=True)
best_Fishers = sorted_Fisher_freq[:number_of_features_all]

best_params_random = []
best_acc_random = []

#all_best_accuracies_MLP = []

for i in range(num_it):
    
    print('Number of Random Selection:' + str(i+1))

    best_Fishers_selected = random.sample(best_Fishers, number_of_features)

    # all_random_selected_features_MLP.append(best_Fishers_selected)

    best_features = []
    best_features_test = []
    best_features_names = []

    for sublist in best_Fishers_selected:
        best_features_names.append((sublist[1]))
        best_features.append(sublist[2])
        best_features_test.append(sublist[3])

    best_features = np.array(best_features)
    best_features_test = np.array(best_features_test)

    # print(best_features.shape)
    # print(best_features_test.shape)

    # print(best_features_names)
    # train_data: (500, 50), train_labels: (550, 1)
    best_features = min_max_normalize(best_features)
    best_features_test = min_max_normalize(best_features_test)

    train_data = best_features_normalize.T
    testing_data = best_features_test.T
    train_labels = training_data_labels.T

# Convert labels from -1 to 0 for compatibility with "binary_crossentropy" loss
    train_labels = (train_labels + 1) / 2

    num_folds = 5
    kf = KFold(n_splits=num_folds)

    parameters = [[1,[32],['relu']],
              [3,[128,64,32],['relu','relu','relu']],
              [3,[32,16,8],['relu','relu','sigmoid']],
              ]

    best_accuracy = 0
    best_params = []

    for params in parameters:
        fold_accuracies = []
        print(f'Parameters: number of layers:{params[0]}, number of neurons in layers:{params[1]}, activation of layers:{params[2]}')

        model = create_mlp_model(params[0],params[1],params[2])
        for train_index, val_index in kf.split(train_data):
            X_train, X_val = train_data[train_index], train_data[val_index]
            y_train, y_val = train_labels[train_index], train_labels[val_index]

            model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=0)

            val_predictions = (model.predict(X_val) > 0.5).astype(int)
            accuracy = accuracy_score(y_val, val_predictions)
            fold_accuracies.append(accuracy)

        average_accuracy = np.mean(fold_accuracies)
        print(f'Average Cross-Validation Accuracy: {average_accuracy * 100:.2f}%')
        print('-' * 30)

        if average_accuracy > best_accuracy:
            best_accuracy = average_accuracy
            best_params = params
        print(best_params)
        print(best_accuracy)

        model.fit(train_data, train_labels, epochs=30, batch_size=32, verbose=0)
        test_predict = (model.predict(testing_data) > 0.5).astype(int)

        all_predicted_MLP.append(test_predict)
        all_best_accuracies_MLP.append(average_accuracy)

    best_params_random.append(best_params)
    best_acc_random.append(best_accuracy)

    #all_best_accuracies_MLP.append(best_accuracy) 
    #all_random_selected_features_MLP.append(best_features)
    

    # print(all_best_params_MLP)  
    # print(all_best_accuracies_MLP)    

    # print(f'Best MLP Network Configuration:')
    # print(f'Number of Layers: {best_params[0]}')
    # print(f'Neurons per Layer: {best_params[1]}')
    # print(f'Activation Function: {best_params[2]}')
    # print(f'Average Cross-Validation Accuracy: {best_accuracy * 100:.2f}%')

Number of Random Selection:1
Parameters: number of layers:1, number of neurons in layers:[32], activation of layers:['relu']
4/4 [==============================] - 0s 491us/step
Average Cross-Validation Accuracy: 76.91%
------------------------------
[1, [32], ['relu']]
0.7690909090909092
5/5 [==============================] - 0s 426us/step
Parameters: number of layers:3, number of neurons in layers:[128, 64, 32], activation of layers:['relu', 'relu', 'relu']
4/4 [==============================] - 0s 524us/step
Average Cross-Validation Accuracy: 90.55%
------------------------------
[3, [128, 64, 32], ['relu', 'relu', 'relu']]
0.9054545454545455
5/5 [==============================] - 0s 481us/step
Parameters: number of layers:3, number of neurons in layers:[32, 16, 8], activation of layers:['relu', 'relu', 'sigmoid']
4/4 [==============================] - 0s 519us/step
Average Cross-Validation Accuracy: 79.64%
------------------------------
[3, [128, 64, 32], ['relu', 'relu', 'relu']]


In [58]:
for i in range(num_it):
    p = best_params_random[i]
    print(f'Num of random selstion:{i}, Number of Layers: {p[0]}, Neurons per Layer: {p[1]}, Activation Function: {p[2]}, Average Cross-Validation Accuracy: {best_acc_random[i] * 100:.2f}')

Num of random selstion:0, Number of Layers: 3, Neurons per Layer: [128, 64, 32], Activation Function: ['relu', 'relu', 'relu'], Average Cross-Validation Accuracy: 90.55
Num of random selstion:1, Number of Layers: 3, Neurons per Layer: [128, 64, 32], Activation Function: ['relu', 'relu', 'relu'], Average Cross-Validation Accuracy: 87.82
Num of random selstion:2, Number of Layers: 3, Neurons per Layer: [128, 64, 32], Activation Function: ['relu', 'relu', 'relu'], Average Cross-Validation Accuracy: 90.73


predicted values using MLP:

In [84]:
print(len(all_best_accuracies_MLP))
print(len(all_predicted_MLP))
max_acc = np.argmax(all_best_accuracies_MLP)
predicted_MLP_Fisher = all_predicted_MLP[max_acc]
predicted_MLP_Fisher = predicted_MLP_Fisher.ravel()
print(predicted_MLP_Fisher)
# p_best_MLP = all_best_params_MLP[max_acc]
# print(p_best_MLP)

16
16
[1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 0 0
 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 1 1
 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1
 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1
 1 1 1 0 0 0 1 1 1 1 1]


# *RBF*

best 50 features using Fisher method

In [60]:
#print(type(fishersOfFeatures_freq[0]))
number_of_features = 50

sorted_Fisher_freq = sorted(fishersOfFeatures, key=lambda x: x[0], reverse=True)
best_Fishers = sorted_Fisher_freq[:number_of_features]

best_features = []
best_features_test = []
best_features_names = []

for sublist in best_Fishers:
    best_features_names.append((sublist[1]))
    best_features.append(sublist[2])
    best_features_test.append(sublist[3])

best_features = np.array(best_features)

best_features_test = np.array(best_features_test)


print(best_features.shape)
print(best_features_test.shape)

#print(best_features_names)

(50, 550)
(50, 159)


In [61]:
best_features_normalize = min_max_normalize(best_features)
best_features_test_normalize = min_max_normalize(best_features_test)
print(best_features_normalize.shape)

(50, 550)


In [62]:
all_best_accuracies_RBF = []
all_predicted_RBF = []

implement RBF network

In [63]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

train_data = best_features_normalize.T
train_labels = training_data_labels.T
tesing_data1 = best_features_test_normalize.T
print(tesing_data1.shape)

# Convert labels from -1 to 0 for compatibility with "binary_crossentropy" loss
train_labels = (train_labels + 1) / 2

train_labels = train_labels.flatten()


svm_model = SVC(kernel='rbf')

# Define the parameters
Cs = [0.1, 1, 10, 15, 18, 100]
gammas = [1, 0.1, 0.01, 0.001]

best_params = {'C': None, 'gamma': None}
best_accuracy = 0.0

num_folds = 5
kf = KFold(n_splits=num_folds)

print("Accuracies for Different Parameter Combinations:")
for C in Cs:
    for gamma in gammas:

        current_model = SVC(kernel='rbf', C=C, gamma=gamma)
        cv_scores = cross_val_score(current_model, train_data, train_labels, cv=5, scoring='accuracy')
        average_accuracy = np.mean(cv_scores)

        current_model.fit(train_data, train_labels)
        test_prediction = current_model.predict(tesing_data1)
        #print(test_prediction.shape)

        all_best_accuracies_RBF.append(average_accuracy)
        all_predicted_RBF.append(test_prediction)

        print(f"Parameters: {{'C': {C}, 'gamma': {gamma}}} - Accuracy: {average_accuracy* 100:.2f}")

        if average_accuracy > best_accuracy:
            best_accuracy = average_accuracy
            best_params['C'] = C
            best_params['gamma'] = gamma

print('-' * 30)
print("\nBest Parameters:", best_params)
print(f'Average Cross-Validation Accuracy: {best_accuracy * 100:.2f}%')

(159, 50)
Accuracies for Different Parameter Combinations:
Parameters: {'C': 0.1, 'gamma': 1} - Accuracy: 60.18
Parameters: {'C': 0.1, 'gamma': 0.1} - Accuracy: 64.00
Parameters: {'C': 0.1, 'gamma': 0.01} - Accuracy: 50.36
Parameters: {'C': 0.1, 'gamma': 0.001} - Accuracy: 50.36
Parameters: {'C': 1, 'gamma': 1} - Accuracy: 78.73
Parameters: {'C': 1, 'gamma': 0.1} - Accuracy: 68.91
Parameters: {'C': 1, 'gamma': 0.01} - Accuracy: 64.73
Parameters: {'C': 1, 'gamma': 0.001} - Accuracy: 50.36
Parameters: {'C': 10, 'gamma': 1} - Accuracy: 81.64
Parameters: {'C': 10, 'gamma': 0.1} - Accuracy: 74.55
Parameters: {'C': 10, 'gamma': 0.01} - Accuracy: 68.36
Parameters: {'C': 10, 'gamma': 0.001} - Accuracy: 65.27
Parameters: {'C': 15, 'gamma': 1} - Accuracy: 80.73
Parameters: {'C': 15, 'gamma': 0.1} - Accuracy: 74.36
Parameters: {'C': 15, 'gamma': 0.01} - Accuracy: 69.64
Parameters: {'C': 15, 'gamma': 0.001} - Accuracy: 65.64
Parameters: {'C': 18, 'gamma': 1} - Accuracy: 80.55
Parameters: {'C': 18,

In [64]:
print(len(all_best_accuracies_RBF))
print(len(all_predicted_RBF))

24
24


randomly selecting 50 of 75 best features (according to Fisher method)

In [65]:
#print(type(fishersOfFeatures_freq[0]))
import random

number_of_features_all = 75
number_of_features = 50
num_it = 4
sorted_Fisher_freq = sorted(fishersOfFeatures, key=lambda x: x[0], reverse=True)
best_Fishers = sorted_Fisher_freq[:number_of_features_all]


best_params_RBF = []
best_accuracies_RBF = []
#all_random_selected_features_RBF = []
for i in range(num_it):
    
    print('Number of Random Selection:' + str(i+1))

    random_Fishers_selected = random.sample(best_Fishers, number_of_features)
    #all_random_selected_features_RBF.append(best_Fishers_selected)


    best_features = []
    best_features_test = []
    best_features_names = []

    for sublist in random_Fishers_selected:
        best_features_names.append((sublist[1]))
        best_features.append(sublist[2])
        best_features_test.append(sublist[3])


    best_features = np.array(best_features)
    best_features_test = np.array(best_features_test)

    best_features = min_max_normalize(best_features)
    best_features_test = min_max_normalize(best_features_test)

    train_data = best_features.T
    testing_data = best_features_test.T
    train_labels = training_data_labels.T

    # Convert labels from -1 to 0 for compatibility with "binary_crossentropy" loss
    train_labels = (train_labels + 1) / 2
    train_labels = train_labels.flatten()

    svm_model = SVC(kernel='rbf')

    # Define the parameters
    Cs = [0.1, 1, 10, 15, 18, 100]
    gammas = [1, 0.1, 0.01, 0.001]

    best_params = []
    best_accuracy = 0.0

    num_folds = 5
    kf = KFold(n_splits=num_folds)

    print("Accuracies for Different Parameter Combinations:")
    for C in Cs:
        for gamma in gammas:

            current_model = SVC(kernel='rbf', C=C, gamma=gamma)
            cv_scores = cross_val_score(current_model, train_data, train_labels, cv=5, scoring='accuracy')
            average_accuracy = np.mean(cv_scores)

            current_model.fit(train_data, train_labels)
            test_prediction = current_model.predict(testing_data)
            #print(test_prediction.shape)

            all_best_accuracies_RBF.append(average_accuracy)
            all_predicted_RBF.append(test_prediction)

            print(f"Parameters: {{'C': {C}, 'gamma': {gamma}}} - Accuracy: {average_accuracy* 100:.2f}")

            if average_accuracy > best_accuracy:
                best_accuracy = average_accuracy
                best_params = [C,gamma]
                #best_params['gamma'] = gamma

  
            # print(best_params)
            # print(best_accuracy)
    #print('-' * 30)
    print("\nBest Parameters:", best_params)
    print(f'Average Cross-Validation Accuracy: {best_accuracy * 100:.2f}%')

    best_params_RBF.append(best_params) 
    best_accuracies_RBF.append(best_accuracy) 
    # print(all_best_params_RBF)  
    # print(all_best_accuracies_RBF)    

    # print(f'Best MLP Network Configuration:')
    # print(f'Number of Layers: {best_params[0]}')
    # print(f'Neurons per Layer: {best_params[1]}')
    # print(f'Activation Function: {best_params[2]}')
    # print(f'Average Cross-Validation Accuracy: {best_accuracy * 100:.2f}%')

# print(len(all_best_params_RBF))

Number of Random Selection:1
Accuracies for Different Parameter Combinations:
Parameters: {'C': 0.1, 'gamma': 1} - Accuracy: 57.27
Parameters: {'C': 0.1, 'gamma': 0.1} - Accuracy: 61.82
Parameters: {'C': 0.1, 'gamma': 0.01} - Accuracy: 50.36
Parameters: {'C': 0.1, 'gamma': 0.001} - Accuracy: 50.36
Parameters: {'C': 1, 'gamma': 1} - Accuracy: 78.91
Parameters: {'C': 1, 'gamma': 0.1} - Accuracy: 69.64
Parameters: {'C': 1, 'gamma': 0.01} - Accuracy: 62.73
Parameters: {'C': 1, 'gamma': 0.001} - Accuracy: 50.36
Parameters: {'C': 10, 'gamma': 1} - Accuracy: 82.00
Parameters: {'C': 10, 'gamma': 0.1} - Accuracy: 74.55
Parameters: {'C': 10, 'gamma': 0.01} - Accuracy: 68.55
Parameters: {'C': 10, 'gamma': 0.001} - Accuracy: 63.45
Parameters: {'C': 15, 'gamma': 1} - Accuracy: 82.00
Parameters: {'C': 15, 'gamma': 0.1} - Accuracy: 74.91
Parameters: {'C': 15, 'gamma': 0.01} - Accuracy: 70.18
Parameters: {'C': 15, 'gamma': 0.001} - Accuracy: 63.45
Parameters: {'C': 18, 'gamma': 1} - Accuracy: 82.00
Pa

In [66]:
for i in range(num_it):
    p = best_params_RBF[i]
    print(f'Num of random selction:{i}, C: {p[0]}, gamma: {p[1]}, Average Cross-Validation Accuracy: {best_accuracies_RBF[i] * 100:.2f}')


Num of random selction:0, C: 10, gamma: 1, Average Cross-Validation Accuracy: 82.00
Num of random selction:1, C: 10, gamma: 1, Average Cross-Validation Accuracy: 81.27
Num of random selction:2, C: 15, gamma: 1, Average Cross-Validation Accuracy: 82.18
Num of random selction:3, C: 10, gamma: 1, Average Cross-Validation Accuracy: 83.09


predicted values using RBF

In [67]:
print(len(all_best_accuracies_RBF))
print(len(all_predicted_RBF))
max_acc = np.argmax(all_best_accuracies_RBF)
predicted_RBF_Fisher = all_predicted_RBF[max_acc]
print(predicted_RBF_Fisher)

120
120
[0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0.]


# Genetic Algorithm 

1000 best features according to Fisher method

In [68]:
#print(type(fishersOfFeatures_freq[0]))
number_of_features_g = 1000

sorted_Fisher_freq_g  = sorted(fishersOfFeatures, key=lambda x: x[0], reverse=True)
best_Fishers_g  = sorted_Fisher_freq_g [:number_of_features_g]

best_features_g  = []
best_features_test_g  = []
best_features_names_g = []

for sublist in best_Fishers_g:
    best_features_names_g.append((sublist[1]))
    best_features_g.append(sublist[2])
    best_features_test_g.append(sublist[3])

best_features_g  = np.array(best_features_g )
best_features_test_g  = np.array(best_features_test_g)
best_features_g_normalize = min_max_normalize(best_features_g)
best_features_test_g_normalize = min_max_normalize(best_features_test_g)

print(best_features_g_normalize.shape)
print(best_features_g_normalize.shape)
print(type(best_features_g))

#print(best_features_names_g)


(1000, 550)
(1000, 550)
<class 'numpy.ndarray'>


In [69]:
X = best_features_g_normalize.T
y = training_data_labels.T
print(X.shape)
print(y.shape)

(550, 1000)
(550, 1)


PSO

In [70]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

class Particle:
    def __init__(self, num_features):
        self.position = np.random.choice([0.0, 1.0], size=num_features)
        self.velocity = np.random.rand(num_features) * 0.15
        self.best_position = self.position.copy()
        self.best_score = float('inf')


def update_velocity(particle, top_position, w=0.05, a=0.01, b=0.01):
    r1, r2 = np.random.rand(2, particle.velocity.shape[0])
    self_velocity = a * r1 * (particle.best_position.astype(np.float32) - particle.position.astype(np.float32))
    neighbor_velocity = b * r2 * (top_position.astype(np.float32) - particle.position.astype(np.float32))
    particle.velocity = w * particle.velocity + self_velocity + neighbor_velocity

def update_position(particle):
    prob = 1 / (1 + np.exp(-particle.velocity))
    particle.position = np.random.rand(len(particle.position)) < prob

def fitness_function(selected_features, X_subset, y):
    num_features_selected = len(selected_features)
    
    clf = RandomForestClassifier(n_estimators=20)
    scores = cross_val_score(clf, X_subset, y, cv=5)
    
    penalty = 0
    ideal_feature_count = 100
    if num_features_selected > ideal_feature_count:
        penalty = abs(num_features_selected - ideal_feature_count) * 500
    
    if num_features_selected == 0:
        return float('inf')

    return -(scores.mean()) + penalty



def pso(X_subset, y, num_features, num_particles=30, iter=50):
    particles = [Particle(num_features) for _ in range(num_particles)]
    top_score = float('inf')
    top_position = np.zeros(num_features)

    for t in range(iter):
        for particle in particles:
            evaluation = fitness_function(
                [i for i, bit in enumerate(particle.position) if bit],
                X_subset,
                y
            )

            if evaluation < top_score:
                top_score = evaluation
                top_position = particle.position.copy()

            if evaluation < particle.best_score:
                particle.best_score = evaluation
                particle.best_position = particle.position.copy()

        for particle in particles:
            update_velocity(particle, top_position)
            update_position(particle)

    return top_position, top_score

# Run PSO
selected_position, _ = pso(X, y.ravel(), number_of_features_g)
selected_features_pso = [i for i, bit in enumerate(selected_position > 0.5) if bit]

print("Selected features:", selected_features_pso)



Selected features: [0, 2, 3, 5, 6, 9, 16, 17, 20, 23, 24, 27, 28, 29, 32, 38, 40, 42, 46, 48, 50, 51, 52, 54, 56, 57, 58, 59, 62, 63, 64, 65, 70, 72, 73, 75, 79, 80, 81, 83, 84, 88, 90, 91, 94, 96, 97, 98, 99, 100, 102, 104, 107, 109, 110, 115, 117, 119, 122, 126, 127, 130, 132, 136, 138, 139, 142, 151, 152, 156, 157, 159, 160, 163, 168, 170, 171, 172, 173, 178, 180, 181, 182, 184, 189, 190, 191, 195, 198, 200, 208, 209, 210, 211, 214, 216, 217, 222, 224, 225, 226, 227, 229, 230, 231, 232, 235, 236, 238, 239, 240, 244, 245, 247, 256, 259, 260, 261, 264, 265, 270, 271, 273, 274, 275, 277, 279, 282, 283, 287, 289, 293, 298, 299, 301, 302, 303, 304, 306, 316, 317, 318, 319, 320, 321, 323, 325, 326, 327, 328, 329, 330, 331, 333, 334, 337, 339, 344, 345, 347, 348, 350, 352, 353, 354, 358, 361, 366, 370, 373, 375, 376, 378, 380, 383, 385, 388, 393, 394, 395, 396, 399, 400, 401, 404, 405, 409, 410, 411, 414, 418, 419, 421, 428, 429, 431, 433, 436, 438, 442, 447, 448, 450, 451, 452, 453, 455, 

In [71]:
print(X.shape)
X_train_PSO = X[:, selected_features_pso]
print("X_train shape:", X_train_PSO.shape)
X_test = best_features_test_g_normalize.T[:, selected_features_pso]
print(X_test.shape)

(550, 1000)
X_train shape: (550, 447)
(159, 447)


## MLP

In [72]:
all_best_accuracies_MLP_PSO = []
all_predicted_MLP_PSO = []

In [73]:
print(X_train_PSO.shape)
train_data = X_train_PSO
train_labels = training_data_labels.T
testing_data = X_test

print(train_data.shape)
print(train_labels.shape)
print(testing_data.shape)

# Convert labels from -1 to 0 for compatibility with "binary_crossentropy" loss
train_labels = (train_labels + 1) / 2

num_folds = 5
kf = KFold(n_splits=num_folds)

parameters = [[1,[32],['relu']],
              [2,[64,32],['relu','relu']],
              [3,[128,64,32],['relu','relu','sigmoid']],
              [3,[128,64,32],['relu','relu','relu']],
              [3,[32,16,8],['relu','relu','sigmoid']],
              [4,[128,63,32,16],['relu','relu','relu','relu']],
              [5,[32,64,128,64,32],['relu','relu','relu','relu','relu']]
              ]

def create_mlp_model(num_layers, neurons_per_layer, activation):
    model = Sequential()
    model.add(Dense(units=neurons_per_layer[0], activation=activation[0], input_dim=train_data.shape[1]))

    for i in range(1, num_layers - 1):
        model.add(Dense(units=neurons_per_layer[i], activation=activation[i]))

    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

best_accuracy = 0

for params in parameters:
    fold_accuracies = []
    model = create_mlp_model(params[0],params[1],params[2])
    print(f'Parameters: number of layers:{params[0]}, number of neurons in layers:{params[1]}, activation of layers:{params[2]}')
    for train_index, val_index in kf.split(train_data):
        X_train, X_val = train_data[train_index], train_data[val_index]
        y_train, y_val = train_labels[train_index], train_labels[val_index]

        
        model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=0)

        val_predictions = (model.predict(X_val) > 0.5).astype(int)
        accuracy = accuracy_score(y_val, val_predictions)
        fold_accuracies.append(accuracy)

    average_accuracy = np.mean(fold_accuracies)
    print(f'Average Cross-Validation Accuracy: {average_accuracy * 100:.2f}%')
    print('-' * 30)


    model.fit(train_data, train_labels, epochs=30, batch_size=32, verbose=0)

    test_predict = (model.predict(testing_data) > 0.5).astype(int)

    #all_best_params_MLP.append(params)
    all_best_accuracies_MLP_PSO.append(average_accuracy)
    all_predicted_MLP_PSO.append(test_predict)

    if average_accuracy > best_accuracy:
        best_accuracy = average_accuracy
        best_params = params


(550, 447)
(550, 447)
(550, 1)
(159, 447)
Parameters: number of layers:1, number of neurons in layers:[32], activation of layers:['relu']
4/4 [==============================] - 0s 497us/step
Average Cross-Validation Accuracy: 90.00%
------------------------------
5/5 [==============================] - 0s 451us/step
Parameters: number of layers:2, number of neurons in layers:[64, 32], activation of layers:['relu', 'relu']
4/4 [==============================] - 0s 475us/step
Average Cross-Validation Accuracy: 93.45%
------------------------------
5/5 [==============================] - 0s 472us/step
Parameters: number of layers:3, number of neurons in layers:[128, 64, 32], activation of layers:['relu', 'relu', 'sigmoid']
4/4 [==============================] - 0s 532us/step
Average Cross-Validation Accuracy: 96.18%
------------------------------
5/5 [==============================] - 0s 500us/step
Parameters: number of layers:3, number of neurons in layers:[128, 64, 32], activation of laye

In [74]:
print(f'Best MLP Network Configuration:')
print(f'Number of Layers: {best_params[0]}')
print(f'Neurons per Layer: {best_params[1]}')
print(f'Activation Function: {best_params[2]}')
print(f'Average Cross-Validation Accuracy: {best_accuracy * 100:.2f}%')

Best MLP Network Configuration:
Number of Layers: 4
Neurons per Layer: [128, 63, 32, 16]
Activation Function: ['relu', 'relu', 'relu', 'relu']
Average Cross-Validation Accuracy: 96.36%


predicted values using MLP and PSO

In [85]:
print(len(all_best_accuracies_MLP_PSO))
print(len(all_predicted_MLP_PSO))
max_acc = np.argmax(all_best_accuracies_MLP_PSO)
predicted_MLP_PSO = all_predicted_MLP_PSO[max_acc]
predicted_MLP_PSO = predicted_MLP_PSO.ravel()
print(predicted_MLP_PSO)

7
7
[0 0 1 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 0 1
 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 1
 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0 1 0 0 0 1 1 0 1 1 1
 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1
 1 1 1 1 0 1 1 0 1 0 1]


# RBF

In [76]:
all_best_accuracies_RBF_PSO = []
all_predicted_RBF_PSO = []

In [77]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

train_data = X_train_PSO
train_labels = training_data_labels.T
testing_data = X_test

print(train_data.shape)
print(train_labels.shape)
print(testing_data.shape)

# Convert labels from -1 to 0 for compatibility with "binary_crossentropy" loss
train_labels = (train_labels + 1) / 2

train_labels = train_labels.flatten()

svm_model = SVC(kernel='rbf')

# Define the parameters
Cs = [0.1, 1, 10, 15, 18, 100]
gammas = [1, 0.1, 0.01, 0.001]

best_params = {'C': None, 'gamma': None}
best_accuracy = 0.0

num_folds = 5
kf = KFold(n_splits=num_folds)

print("Accuracies for Different Parameter Combinations:")
for C in Cs:
    for gamma in gammas:

        current_model = SVC(kernel='rbf', C=C, gamma=gamma)
        cv_scores = cross_val_score(current_model, train_data, train_labels, cv=5, scoring='accuracy')
        average_accuracy = np.mean(cv_scores)

        current_model.fit(train_data, train_labels)
        test_prediction = current_model.predict(testing_data)
        #print(test_prediction.shape)

        all_best_accuracies_RBF_PSO.append(average_accuracy)
        all_predicted_RBF_PSO.append(test_prediction)

        print(f"Parameters: {{'C': {C}, 'gamma': {gamma}}} - Accuracy: {average_accuracy* 100:.2f}")

        if average_accuracy > best_accuracy:
            best_accuracy = average_accuracy
            best_params['C'] = C
            best_params['gamma'] = gamma

print('-' * 30)
print("\nBest Parameters:", best_params)
print(f'Average Cross-Validation Accuracy: {best_accuracy * 100:.2f}%')

(550, 447)
(550, 1)
(159, 447)
Accuracies for Different Parameter Combinations:
Parameters: {'C': 0.1, 'gamma': 1} - Accuracy: 50.36
Parameters: {'C': 0.1, 'gamma': 0.1} - Accuracy: 50.91
Parameters: {'C': 0.1, 'gamma': 0.01} - Accuracy: 50.36
Parameters: {'C': 0.1, 'gamma': 0.001} - Accuracy: 50.36
Parameters: {'C': 1, 'gamma': 1} - Accuracy: 61.45
Parameters: {'C': 1, 'gamma': 0.1} - Accuracy: 85.27
Parameters: {'C': 1, 'gamma': 0.01} - Accuracy: 71.09
Parameters: {'C': 1, 'gamma': 0.001} - Accuracy: 51.09
Parameters: {'C': 10, 'gamma': 1} - Accuracy: 64.00
Parameters: {'C': 10, 'gamma': 0.1} - Accuracy: 88.91
Parameters: {'C': 10, 'gamma': 0.01} - Accuracy: 83.82
Parameters: {'C': 10, 'gamma': 0.001} - Accuracy: 73.45
Parameters: {'C': 15, 'gamma': 1} - Accuracy: 64.00
Parameters: {'C': 15, 'gamma': 0.1} - Accuracy: 88.91
Parameters: {'C': 15, 'gamma': 0.01} - Accuracy: 85.45
Parameters: {'C': 15, 'gamma': 0.001} - Accuracy: 77.45
Parameters: {'C': 18, 'gamma': 1} - Accuracy: 64.00


predicted values using RBF and PSO

In [78]:
print(len(all_best_accuracies_RBF_PSO))
print(len(all_predicted_RBF_PSO))
max_acc = np.argmax(all_best_accuracies_RBF_PSO)
predicted_RBF_PSO = all_predicted_RBF_PSO[max_acc]
print(predicted_RBF_PSO)

24
24
[0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0.
 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1.]


In [86]:
print(type(predicted_MLP_Fisher))
print(type(predicted_RBF_Fisher))
print(type(predicted_MLP_PSO))
print(type(predicted_RBF_PSO))

print(predicted_MLP_Fisher.shape)
print(predicted_RBF_Fisher.shape)
print(predicted_MLP_PSO.shape)
print(predicted_RBF_PSO.shape)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(159,)
(159,)
(159,)
(159,)


In [93]:
predicted_MLP_Fisher_final = np.where(predicted_MLP_Fisher == 0, -1 , predicted_MLP_Fisher)
predicted_RBF_Fisher_final = np.where(predicted_RBF_Fisher == 0, -1 , predicted_RBF_Fisher)
predicted_MLP_PSO_final = np.where(predicted_MLP_PSO == 0, -1 , predicted_MLP_PSO)
predicted_RBF_PSO_final = np.where(predicted_RBF_PSO == 0, -1 , predicted_RBF_PSO)


In [98]:
print(predicted_MLP_Fisher_final)
print(predicted_RBF_Fisher_final)
print(predicted_MLP_PSO_final)
print(predicted_RBF_PSO_final)

[ 1 -1  1  1  1 -1  1 -1  1  1 -1  1  1 -1  1 -1  1  1  1 -1 -1 -1  1 -1
  1  1  1  1 -1  1 -1  1 -1 -1  1 -1 -1  1  1  1  1  1 -1  1  1 -1  1  1
  1  1  1  1  1  1 -1 -1  1 -1  1  1 -1 -1  1  1  1 -1  1  1 -1 -1  1 -1
  1  1  1  1  1 -1  1  1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1 -1 -1  1
  1  1  1 -1 -1  1 -1 -1  1  1  1 -1  1 -1  1  1 -1  1  1 -1  1  1  1  1
  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1 -1  1  1  1  1  1 -1  1
  1 -1 -1  1  1  1  1 -1 -1 -1  1  1  1  1  1]
[-1. -1.  1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1.  1. -1. -1.  1. -1.
 -1. -1. -1. -1.  1. -1. -1. -1. -1.  1. -1.  1. -1.  1. -1. -1. -1. -1.
 -1.  1.  1.  1.  1.  1. -1.  1. -1. -1.  1. -1.  1.  1.  1.  1.  1. -1.
 -1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1. -1. -1. -1. -1.
  1.  1. -1.  1.  1. -1.  1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1.
  1. -1.  1.  1. -1.  1. -1. -1.  1. -1. -1.  1.  1. -1.  1.  1.  1.  1.
 -1.

In [99]:
print(predicted_MLP_Fisher)
print(predicted_RBF_Fisher)
print(predicted_MLP_PSO)
print(predicted_RBF_PSO)

[1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 0 0
 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 1 1
 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 0 1 0 1
 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1
 1 1 1 0 0 0 1 1 1 1 1]
[0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0.]
[0 0 1 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 1 0 0 0 1 1 1 1 0 1 0 0 0 0 1
 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 1
 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 

In [95]:
import scipy.io
scipy.io.savemat('Test_MLP1.mat', {'Test_MLP1': predicted_MLP_Fisher_final})
scipy.io.savemat('Test_RBF1.mat', {'Test_RBF1': predicted_RBF_Fisher_final})
scipy.io.savemat('Test_MLP2.mat', {'Test_MLP2': predicted_MLP_PSO_final})
scipy.io.savemat('Test_RBF2.mat', {'Test_RBF2': predicted_RBF_PSO_final})

#### The End